In [1]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np


In [2]:
Train_dir = 'train'
Test_dir = 'test'

In [3]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in sorted(os.listdir(dir)):
        label_path = os.path.join(dir, label)
        if not os.path.isdir(label_path):
            continue  # skip files like .DS_Store
        for imagename in os.listdir(label_path):
            image_paths.append(os.path.join(label_path, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [4]:
train = pd.DataFrame()
train["image"], train["label"] = createdataframe(Train_dir)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [5]:
print(train)

                          image     label
0         train/angry/14147.jpg     angry
1         train/angry/24084.jpg     angry
2           train/angry/823.jpg     angry
3         train/angry/19463.jpg     angry
4         train/angry/30797.jpg     angry
...                         ...       ...
28816   train/surprise/7075.jpg  surprise
28817  train/surprise/11740.jpg  surprise
28818   train/surprise/4568.jpg  surprise
28819  train/surprise/18764.jpg  surprise
28820   train/surprise/6419.jpg  surprise

[28821 rows x 2 columns]


In [6]:
test = pd.DataFrame()
test["image"], test["label"] = createdataframe(Test_dir) 

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [7]:
print(test)

                        image     label
0        test/angry/19477.jpg     angry
1        test/angry/29588.jpg     angry
2        test/angry/11017.jpg     angry
3        test/angry/35162.jpg     angry
4         test/angry/4015.jpg     angry
...                       ...       ...
7061   test/surprise/5138.jpg  surprise
7062   test/surprise/6425.jpg  surprise
7063  test/surprise/25397.jpg  surprise
7064   test/surprise/2119.jpg  surprise
7065   test/surprise/1376.jpg  surprise

[7066 rows x 2 columns]


In [8]:
from tqdm.notebook import tqdm

In [9]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale', target_size=(48, 48))
        img = np.array(img) 
        features.append(img)
    features = np.array(features) ## neural networks (like Keras/TensorFlow) expect a single NumPy array
    features = features.reshape(len(features), 48,48,1)
    return features

In [10]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [11]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [12]:
x_train = train_features/255.0
x_test = test_features/255.0

In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [15]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])


In [34]:
y_train = to_categorical(y_train, num_classes = 7)
y_test = to_categorical(y_test, num_classes = 7)


In [ ]:
model = Sequential()

model.add(Conv2D,(128, kernel_size=()))